In [ ]:
import os
from google.colab import drive
import numpy as np
import tensorflow.keras as krs
from keras.datasets import mnist
from PIL import Image, ImageFilter, ImageOps
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/sukhanich-lapochka')

Mounted at /content/drive


In [ ]:
save_dir = 'images/assets/'
image_dir = 'images/'

In [ ]:
# @title Текст заголовка по умолчанию
def convert_im(path, save_path):
    color_tolerance = 45  # Color tolerance

    img = Image.open(path)
    img = img.convert('RGB')  # Convert to RGB mode if not already
    arr = np.array(img)
    height = len(arr)  # Height
    width = len(arr[0])  # Width

    # Calculate average RGB value
    aver_rgb_sum = 0
    for i in range(height):
        for j in range(width):
            r, g, b = arr[i, j]
            aver_rgb_sum += r
            aver_rgb_sum += g
            aver_rgb_sum += b

    aver_rgb_sum /= height * width * 3
    color_tolerance = aver_rgb_sum

    for i in range(height):
        for j in range(width):
            cur_sum = sum(arr[i, j]) / 3  # Calculate average intensity
            if cur_sum > color_tolerance / 1.7:
                arr[i, j] = [255, 255, 255]
            else:
                arr[i, j] = [0, 0, 0]

    arr = np.array(Image.fromarray(arr))
    img = ImageOps.invert(Image.fromarray(arr)).resize((28, 28))  # Resize image to 28x28 and invert colors
    img = img.convert('L')  # Convert to grayscale

    img.save(save_path)
    arr = np.array(img)
    return arr

In [ ]:
for i in range(10):
    class_dir = os.path.join(save_dir, str(i))
    os.makedirs(class_dir, exist_ok=True)  # Ensure directory exists for the current class

    # Iterate through images in each class
    for filename in os.listdir(os.path.join(image_dir, str(i))):
        img_path = os.path.join(image_dir, str(i), filename)
        save_path = os.path.join(class_dir, filename)

        # Convert and save the image
        convert_im(img_path, save_path)

In [ ]:
# # Load MNIST dataset

# Define encoder model
nw = nh = 28
num_hide = 98
encoder = krs.models.load_model("encoder_model.h5")
# out_vec = encoder.predict(all_image)

# Define classifier model
num_classes = 10
classificator = krs.models.load_model("classificator_model.h5")

# Load trained weights

# Directory containing the images
true_labels = []
predicted_probs_list = [[] for _ in range(10)]  # List of empty lists for each class
accuracies_per_label = []

# Iterate through each class (0 to 9)
for i in range(10):
    correct_count = 0
    total_count = 0

    # Iterate through images in each class
    for filename in os.listdir(os.path.join(save_dir, str(i))):
        img_path = os.path.join(save_dir, str(i), filename)
        x = [np.array(Image.open(img_path))]  # This returns a list of length 1
        eps = np.array(x).reshape(-1, 28, 28, 1)  # Reshape to match the input shape expected by the encoder
        out_vec = encoder.predict(eps)  # Predict using the encoder model
        predictions = classificator.predict(out_vec)
        predicted_label = np.argmax(predictions)

        true_labels.append(i)
        predicted_probs_list[i].append(predictions[0])  # Append predicted probabilities for this image and class

        # Check if prediction is correct
        if predicted_label == i:
            correct_count += 1
        total_count += 1

        print(f"Image: {img_path}, Predicted Label: {predicted_label}")

    # Calculate accuracy for this label
    accuracy = correct_count / total_count if total_count > 0 else 0
    accuracies_per_label.append(accuracy)

# Print accuracy per label
for i, accuracy in enumerate(accuracies_per_label):
    print(f"Accuracy for label {i}: {accuracy * 100}%")

# Concatenate true labels and predicted probabilities for all classes
true_labels = np.array(true_labels)
predicted_probs = np.concatenate(predicted_probs_list, axis=0)

# Calculate overall accuracy
predicted_labels = np.argmax(predicted_probs, axis=1)
overall_accuracy = np.mean(true_labels == predicted_labels)
print(f"Overall Accuracy: {overall_accuracy * 100}%")

<ipython-input-3-4feafff7a1e3>:18: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = im.resize((nwidth, 20), Image.ANTIALIAS).filter(ImageFilter.SHARPEN)


1/1 [==============================] - 1s 717ms/step
Image: images//0/test_image8.jpg, Predicted Label: 8
1/1 [==============================] - 0s 19ms/step
Image: images//0/test_image6.jpg, Predicted Label: 7
1/1 [==============================] - 0s 19ms/step
Image: images//0/test_image4.jpg, Predicted Label: 8
1/1 [==============================] - 0s 33ms/step
Image: images//0/test_image5.jpg, Predicted Label: 7
1/1 [==============================] - 0s 28ms/step
Image: images//0/test_image0.jpg, Predicted Label: 8
1/1 [==============================] - 0s 28ms/step
Image: images//0/test_image2.jpg, Predicted Label: 7
1/1 [==============================] - 0s 34ms/step
Image: images//0/test_image3.jpg, Predicted Label: 1
1/1 [==============================] - 0s 31ms/step
Image: images//0/test_image7.jpg, Predicted Label: 9
1/1 [==============================] - 0s 19ms/step
Image: images//0/test_image1.jpg, Predicted Label: 8
1/1 [==============================] - 0s 21ms/step
Im

<ipython-input-3-4feafff7a1e3>:11: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img = im.resize((20, nheight), Image.ANTIALIAS).filter(ImageFilter.SHARPEN)


Image: images//2/test_image0.jpg, Predicted Label: 7
1/1 [==============================] - 0s 20ms/step
Image: images//2/test_image9.jpg, Predicted Label: 1
1/1 [==============================] - 0s 19ms/step
Image: images//2/test_image6.jpg, Predicted Label: 7
1/1 [==============================] - 0s 20ms/step
Image: images//2/test_image1.jpg, Predicted Label: 1
1/1 [==============================] - 0s 19ms/step
Image: images//2/test_image10.jpg, Predicted Label: 7
1/1 [==============================] - 0s 30ms/step
Image: images//2/test_image8.jpg, Predicted Label: 7
1/1 [==============================] - 0s 27ms/step
Image: images//2/test_image3.jpg, Predicted Label: 7
1/1 [==============================] - 0s 27ms/step
Image: images//2/test_image7.jpg, Predicted Label: 7
1/1 [==============================] - 0s 31ms/step
Image: images//3/test_image10.jpg, Predicted Label: 7
1/1 [==============================] - 0s 36ms/step
Image: images//3/test_image1.jpg, Predicted Label: 1


In [ ]:
# Plot ROC curve
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(10):
    fpr[i], tpr[i], _ = roc_curve((true_labels == i).astype(int), predicted_probs[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

plt.figure()
for i in range(10):
    plt.plot(fpr[i], tpr[i], label=f'ROC curve (class {i}) (area = {roc_auc[i]:.2f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()